In [53]:
# = = = Inital Setup = = = #
# ======================== #
# After extracting the data, transform it so that it's more usable #
# ======================== #

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 10) # make sure we can display enough columns

# Open our team stats and analytics tables
team_stats_df = pd.read_csv('book8.csv') 
analytics_df = pd.read_csv('analytics.csv')

# Create a new dataframe only with the columns that we want
new_team_df = team_stats_df[["Team", "GP", "GF/G", "GA/G"]].copy()
new_analytic_df = analytics_df[["Team", "xGF", "xGA", "aGF", "aGA"]].copy() # create a new frame only with the columns that we want

# Combine tables 
new_df = pd.merge(new_team_df, new_analytic_df, on='Team') 

# Add columns to our table using a calculation
new_df['5on5_xGF-game'] = new_df.apply(lambda row: row['xGF'] / row["GP"], axis=1) # Use # of xGF / games played to get a per game statistic
new_df['5on5_xGA-game'] = new_df.apply(lambda row: row['xGA'] / row["GP"], axis=1) # Use # of xGA / games played to get a per game statistic
new_df['5on5_aGF-game'] = new_df.apply(lambda row: row['aGF'] / row["GP"], axis=1) # Use # of xGF / games played to get a per game statistic
new_df['5on5_aGA-game'] = new_df.apply(lambda row: row['aGA'] / row["GP"], axis=1) # Use # of xGA / games played to get a per game statistic

new_df = new_df.rename(columns={"GF/G": "TeamGF-game", "GA/G": "TeamGA-game"}).drop(columns=["xGF", "xGA", "aGF", "aGA"]) # Rename some columns and drop cumulative stats

new_df.loc['mean'] = new_df.mean(numeric_only=True) # Create a new row where we call mean on each series.  The index is mean 
new_df.loc['mean', 'Team'] = "Avg" # Update the 'Team' value for our mean row to be Avg

# Separate our models from eachother
team_df = new_df[["Team", "TeamGF-game", "TeamGA-game"]].rename(columns={"TeamGF-game": "GFperGame", "TeamGA-game": "GAperGame"}).round(2)
exp_5on5_df = new_df[["Team", "5on5_xGF-game", "5on5_xGA-game"]].rename(columns={"5on5_xGF-game": "GFperGame", "5on5_xGA-game": "GAperGame"}).round(2)
act_5on5_df = new_df[["Team", "5on5_aGF-game", "5on5_aGA-game"]].rename(columns={"5on5_aGF-game": "GFperGame", "5on5_aGA-game": "GAperGame"}).round(2)

# Add attack and defensive strengths to each model
team_df['AttStr'] = (team_df['GFperGame'] / team_df.loc['mean', 'GFperGame']).round(2)
team_df['DefStr'] = (team_df['GAperGame'] / team_df.loc['mean', 'GAperGame']).round(2)

exp_5on5_df['AttStr'] = (exp_5on5_df['GFperGame'] / exp_5on5_df.loc['mean', 'GFperGame']).round(2)
exp_5on5_df['DefStr'] = (exp_5on5_df['GAperGame'] / exp_5on5_df.loc['mean', 'GAperGame']).round(2)

act_5on5_df['AttStr'] = (act_5on5_df['GFperGame'] / act_5on5_df.loc['mean', 'GFperGame']).round(2)
act_5on5_df['DefStr'] = (act_5on5_df['GAperGame'] / act_5on5_df.loc['mean', 'GAperGame']).round(2)

team_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/models/teamstats.csv', index=False)
exp_5on5_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/models/expectedanalytics.csv', index=False)
act_5on5_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/models/actualanalytics.csv', index=False)

,Team,GFperGame,GAperGame,AttStr,DefStr
0,Boston Bruins,2.43,1.38,1.19,0.68
1,Carolina Hurricanes,2.21,1.57,1.08,0.77
2,New Jersey Devils,2.41,1.71,1.18,0.84
3,Toronto Maple Leafs,2.28,1.70,1.11,0.83
4,Tampa Bay Lightning,2.27,1.88,1.11,0.92
5,New York Rangers,2.13,1.90,1.04,0.93
6,Vegas Golden Knights,2.19,1.98,1.07,0.97
7,Los Angeles Kings,2.02,2.11,0.99,1.03
8,Dallas Stars,2.03,1.64,0.99,0.80
9,Minnesota Wild,1.62,1.73,0.79,0.85


In [57]:
# Note: When you saved these dataframes as csv's in your project's models directory; we were using data from Book8 and analytics.  We may want to use updated ones
# We'll also need a way to update these files, but that should be easy since it's just manipulating the same data and way as above; and then loading it back into each model
team_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/models/teamstats.csv', index=False)
exp_5on5_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/models/expectedanalytics.csv', index=False)
act_5on5_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/models/actualanalytics.csv', index=False)

In [3]:
# !!GOLD

import pandas as pd
from datetime import date

# Turn our schedule into a dataframe that we can manipulate.  Leave out everything but the data I care about (Who's playing and when?)
schedule_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/schedule.csv')
schedule_df = schedule_df[['Date', 'Visitor', 'Home']]

# Turn today's date into a string that we can compare with the values in our schedule data.  Return a string to be compared with the dates in our schedule
def TodaysDateToString():
    pieces = str(date.today()).split('-')
    yyyy = pieces[0]
    mm = pieces[1]
    dd = pieces[2]
    date_today = [mm.lstrip('0'), dd.lstrip('0'), yyyy[-2] + yyyy[-1]]
    comparison_val = "/".join(date_today)
    return comparison_val

# Capture the date and visiting + home teams from each of today's games.  Return a list of matchup dictionaries
def CaptureTodaysGames():
    today = TodaysDateToString() # grab today as a string
    filt = ( schedule_df['Date'] == today ) # define a filter to apply to our schedule that looks for today's date
        
    todays_matchups = [] # empty list of matchups that will be filled and returned
    for index, row in schedule_df.loc[filt].iterrows(): # apply the filter to our dataframe and iterate over the rows
        # append to our list of matchups today, a dictionary for each one, containing the necessary information
        todays_matchups.append({
            'date': row['Date'],
            'visitor': row['Visitor'],
            'home': row['Home']
        })
    return todays_matchups

# ! ! ! May need to make a separate CaptureTodaysGames to grab from predictions
def CaptureTodaysPredictedGames():
    predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')
    today = TodaysDateToString()

    filt = ( predi_df['Date'] == today )

    todays_matchups = [] # empty list of matchups that will be filled and returned
    for index, row in predi_df.loc[filt].iterrows(): # apply the filter to our dataframe and iterate over the rows
        # append to our list of matchups today, a dictionary for each one, containing the necessary information
        todays_matchups.append({
            'date': row['Date'],
            'visitor': row['AwayTeam'],
            'home': row['HomeTeam'],
            'away_chance': row['away_ml_chance'],
            'home_chance': row['home_ml_chance']
        })
    return todays_matchups

# Calculate expected goals for both teams, for all three models.  Return a dictionary with all of the predicted scores
def xG_Calculation(visitor, home):

    # Instantiate our model dataframes and set the index column to team names
    team_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/models/teamstats.csv', index_col='Team')
    exp_5on5_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/models/expectedanalytics.csv', index_col='Team')
    act_5on5_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/models/actualanalytics.csv', index_col='Team')

    # = = = Team Stats Model = = = #
    team_visitor_xg = team_df.loc[visitor, 'AttStr'] * team_df.loc[home, 'DefStr'] * team_df.loc['Avg', 'GFperGame']
    team_home_xg = team_df.loc[home, 'AttStr'] * team_df.loc[visitor, 'DefStr'] * team_df.loc['Avg', 'GFperGame']
    
    # = = = 5on5 Expected Model = = = #
    exp_5on5_visitor_xg = exp_5on5_df.loc[visitor, 'AttStr'] * exp_5on5_df.loc[home, 'DefStr'] * exp_5on5_df.loc['Avg', 'GFperGame']
    exp_5on5_home_xg = exp_5on5_df.loc[home, 'AttStr'] * exp_5on5_df.loc[visitor, 'DefStr'] * exp_5on5_df.loc['Avg', 'GFperGame']

    # = = = 5on5 Actual Model
    act_5on5_visitor_xg = act_5on5_df.loc[visitor, 'AttStr'] * act_5on5_df.loc[home, 'DefStr'] * act_5on5_df.loc['Avg', 'GFperGame']
    act_5on5_home_xg = act_5on5_df.loc[home, 'AttStr'] * act_5on5_df.loc[visitor, 'DefStr'] * act_5on5_df.loc['Avg', 'GFperGame']

    return { 
        'teamstats':{'visitor_xg': team_visitor_xg, 'home_xg': team_home_xg},
        'exp_5on5':{'visitor_xg': exp_5on5_visitor_xg, 'home_xg': exp_5on5_home_xg},
        'act_5on5':{'visitor_xg': act_5on5_visitor_xg, 'home_xg': act_5on5_home_xg} 
        }

# Update predictions database with today's matchups (date/away/home) and expected goals for each team
def Make_Scoring_Predictions_Today():

    # Instantiate our predictions dataframe to be updated with all of today's matchups(date, teams, xGoals, odds)
    predictions_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')

    # - Model Attribution - # - How important do you think each model is?
    teamstat_weight = 0.333
    exp_5on5_weight = 0.333
    act_5on5_weight = 0.333

    # Check predictions to see if we've already ran today's game
    today = TodaysDateToString()
    if today in predictions_df['Date'].values:
        return print("Today's predictions have already been made")
    else:
        print("Capturing today's matchups...")

    # Predict Overall Expected Goals for each team
    for matchup in CaptureTodaysGames(): # iterate over each of today's games
        
        xGoals_Raw = xG_Calculation(matchup['visitor'], matchup['home']) # predictions for each of our 3 models
        
        # Calculate the overall expected goals for both teams based on how much you attribute to each model
        home_xg_ovr = ( ( xGoals_Raw['teamstats']['home_xg'] * teamstat_weight ) + ( xGoals_Raw['exp_5on5']['home_xg'] * exp_5on5_weight ) + ( xGoals_Raw['act_5on5']['home_xg'] * act_5on5_weight ) )
        visitor_xg_ovr = ( ( xGoals_Raw['teamstats']['visitor_xg'] * teamstat_weight ) + ( xGoals_Raw['exp_5on5']['visitor_xg'] * exp_5on5_weight ) + ( xGoals_Raw['act_5on5']['visitor_xg'] * act_5on5_weight ) )

        #print("Away |", matchup['visitor'], ":", visitor_xg_ovr.round(2))
        #print("Home |", matchup['home'], ":", home_xg_ovr.round(2))

        # Add a row to our predictions dataframe with the matchup data and expected goals

        # Create a Dataframe with 1 row for this matchup, to be added to our predictions dataframe
        pred_upd_df = pd.DataFrame({'Date': [matchup['date']], 'AwayTeam': [matchup['visitor']], 'Away_xG': [visitor_xg_ovr.round(2)], 'HomeTeam': [matchup['home']], 'Home_xG': [home_xg_ovr.round(2)]})
        predictions_df = pd.concat([predictions_df, pred_upd_df], ignore_index=True)
        
    # Save our predictions to our predictions database
    predictions_df = predictions_df.drop(columns=['Unnamed: 0'])
    predictions_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv', index=False)
    print("= =========================== =")
    print(" = = = Predictions saved = = = ")
    print("= =========================== =")


In [43]:
# !! USED | WILL NOT NEED AGAIN (unless for reset) !!
import pandas as pd
# Create a DataFrame that will house your predictions, we can have a separate one for vegas odds and we'll merge the two when it comes time
pred = {
    'Date':[], # matchup data
    'AwayTeam':[], # matchup data
    'Away_xG':[], # calculated - models + scoring algorithm
    'Away_mlPred':[], # calculated - poisson + xGoals + american odds algorithm
    'away_ml_chance':[], # calculated - poisson + xGoals
    'Away_vegasPred':[], # odds API
    'HomeTeam':[], # matchup data
    'Home_xG':[], # calculated - models + scoring algorithm
    'Home_mlPred':[], # calculated - poisson + xGoals + american odds algorithm
    'home_ml_chance':[], # calculated - poisson + xGoals
    'Home_vegasPred':[], # odds API
}

pred_df = pd.DataFrame(pred) # Create our dataframe from the columns we laid out above

pred_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv', index=False) # Create a csv with our columns to store our predictions
pred_df

,Date,AwayTeam,Away_xG,Away_mlPred,away_ml_chance,Away_vegasPred,HomeTeam,Home_xG,Home_mlPred,home_ml_chance,Home_vegasPred


In [90]:
# !!TESTING
predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')
matchup = {'Date':'3/2/23', 'AwayTeam':'Washington Capitals', 'HomeTeam':'Anaheim Ducks'}
today = TodaysDateToString()
home_filt = ( (predi_df['HomeTeam'] == matchup['HomeTeam']) & (predi_df['Date'] == today ) )
#home_filt = ( (predi_df['HomeTeam'] == matchup['HomeTeam']) & (predi_df['Date'] == matchup['Date']) ) this was for testing purposes to ensure that my & condition was working
predi_df[home_filt]

away_xg = predi_df[home_filt].loc[0, 'Away_xG']
print(away_xg)

3.24


In [42]:
predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')
predi_df

,Date,AwayTeam,Away_xG,Away_mlPred,Away_vegasPred,HomeTeam,Home_xG,Home_mlPred,Home_vegasPred
0,3/6/23,Edmonton Oilers,3.15,NaN,NaN,Buffalo Sabres,2.88,NaN,NaN
1,3/6/23,Ottawa Senators,2.65,NaN,NaN,Chicago Blackhawks,1.98,NaN,NaN
2,3/6/23,Calgary Flames,2.15,NaN,NaN,Dallas Stars,2.32,NaN,NaN
3,3/6/23,Washington Capitals,2.41,NaN,NaN,Los Angeles Kings,2.43,NaN,NaN
4,3/6/23,Nashville Predators,2.78,NaN,NaN,Vancouver Canucks,2.45,NaN,NaN
5,3/6/23,San Jose Sharks,2.20,NaN,NaN,Winnipeg Jets,2.55,NaN,NaN


In [4]:
# !!GOLD
from scipy.stats import poisson
import pandas as pd

# Update today's predictions with moneyline odds | MUST RUN CALCULATE SCORING PREDICITONS BEFORE THIS OR I SHOULD PUT IT IN IT
def Calculate_Chances():
    pssn_table_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/poissontable.csv').drop(columns=['num']) # Load in our blank poisson table dataframe
    predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv') # load our predictions data
    #print(predi_df)
    predi_df.loc[:, ['home_ml_chance', 'away_ml_chance']] = predi_df.loc[:, ['home_ml_chance', 'away_ml_chance']].fillna(0.000) # make sure our chances are float values we can update with decimal chance of winning
    today = TodaysDateToString()

    # Update predictions database with today's matchups (date/away/home) and expected goals for each team
    Make_Scoring_Predictions_Today()

    # Check to see if we've already calculated today's games
    filt = ( predi_df['Date'] == today )
    if predi_df.loc[filt, 'home_ml_chance'].values.any() > 0:
        return print("You've already calculated moneyline chances for today's games!")

    for matchup in CaptureTodaysPredictedGames():
        # Query our predictions dataframe on both of the teams and the date so that we can grab expected goals for both teams, to be used in our poisson calculation
        home_filt = ( (predi_df['HomeTeam'] == matchup['home']) & (predi_df['Date'] == today ) ) # create a query for the matchup's home team and today's date
        away_filt = ( (predi_df['AwayTeam'] == matchup['visitor']) & (predi_df['Date'] == today ) ) # create a query for the matchup's away team and today's date
        away_xg = predi_df.loc[away_filt, 'Away_xG'] # query away exp goals
        home_xg = predi_df.loc[home_filt, 'Home_xG'] # query home exp goals

        # Fill our poisson table dataframe with data based on the likelihood of home/away team score and expected goal combination occurences
        for i in range(11): # this will be our row value, it will represent the away team's score
            for l in range(11): # this will be our column value, it will represent the home team's score
                
                # for each cell in my dataframe, I want to set it to the likelihood of both team's score outcomes combined
                pssn_table_df.loc[i, f'{l}'] = ( poisson.pmf(k=i, mu=away_xg) ).item() * ( poisson.pmf(k=l, mu=home_xg) ).item()
                chance = pssn_table_df.loc[i, f'{l}'].round(3)
                
                # Create conditions so that we add to each team's likelihood of winning
                # ! not incorporating puckline yet because it will require me to determine whose moneyline chances are greater/less than .500 in order to determine who gets +1.5 and who gets -1.5
                # ! That would really benefit from a programmatic way of scraping a poisson table
                if i == l:
                    predi_df.loc[home_filt, 'home_ml_chance'] += (0.5 * chance)
                    predi_df.loc[home_filt, 'away_ml_chance'] += (0.5 * chance)
                elif l > i:
                    predi_df.loc[home_filt, 'home_ml_chance'] += chance
                elif l < i:
                    predi_df.loc[home_filt, 'away_ml_chance'] += chance
        
        # Save predicted % chances
        predi_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv', index=False)
    return print("Moneyline chance predictions saved!")

#Make_Scoring_Predictions_Today()
Calculate_Chances()
predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')
#predi_df[['Date', 'AwayTeam', 'Away_xG', 'away_ml_chance', 'HomeTeam', 'Home_xG', 'home_ml_chance']]
today = TodaysDateToString()
filt = ( predi_df['Date'] == today )
#predi_df.loc[filt, ['Date', 'AwayTeam', 'Away_xG', 'away_ml_chance', 'HomeTeam', 'Home_xG', 'home_ml_chance'] ]
predi_df[filt]

# NEED TO BUILD IN ML CHANCE -> PRED ODDS METHOD

Today's predictions have already been made
Moneyline chance predictions saved!


,Date,AwayTeam,Away_xG,Away_mlPred,away_ml_chance,Away_vegasPred,HomeTeam,Home_xG,Home_mlPred,home_ml_chance,Home_vegasPred
0,3/6/23,Edmonton Oilers,3.20,NaN,0.5560,NaN,Buffalo Sabres,2.85,NaN,0.4430,NaN
1,3/6/23,Ottawa Senators,2.77,NaN,0.6495,NaN,Chicago Blackhawks,1.89,NaN,0.3455,NaN
2,3/6/23,Calgary Flames,2.14,NaN,0.4515,NaN,Dallas Stars,2.39,NaN,0.5445,NaN
3,3/6/23,Washington Capitals,2.49,NaN,0.5055,NaN,Los Angeles Kings,2.45,NaN,0.4925,NaN
4,3/6/23,Nashville Predators,2.71,NaN,0.5490,NaN,Vancouver Canucks,2.41,NaN,0.4470,NaN
5,3/6/23,San Jose Sharks,2.23,NaN,0.4160,NaN,Winnipeg Jets,2.68,NaN,0.5750,NaN


In [224]:
predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')
predi_df.loc[:, ['Away_mlPred', 'Home_mlPred']]

,Away_mlPred,Home_mlPred
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0


In [54]:
# Take today's predicted games % chance and calculate american odds for each team
import numpy as np

def CalcMLOdds():
    
    predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')

    predi_df.loc[:, ['Away_mlPred', 'Home_mlPred']] = predi_df.loc[:, ['Away_mlPred', 'Home_mlPred']].fillna(0.000) # make sure our odds are float values we can update with decimal chance of winning

    today = TodaysDateToString()
    filt = ( predi_df['Date'] == today)
    

    # if we've already calculated today's moneyline odds, return
    if predi_df.loc[filt, 'Home_mlPred'].values.any() > 0:
        return print("You've already calcaulted moneyline odds for today's games!")

    # run predictedgamestoday function which also captures away/home ml % chances

    for matchup in CaptureTodaysPredictedGames():

        home_filt = ( ( predi_df['Date'] == today) & ( predi_df['HomeTeam'] == matchup['home'] ) )
        away_filt = ( ( predi_df['Date'] == today) & ( predi_df['AwayTeam'] == matchup['visitor'] ) )

        if matchup['home_chance'] > 0.5: # is the home team the favorite?
            predi_df.loc[home_filt, 'Home_mlPred'] = np.round( (-1) * ( 100 / ( (1/matchup['home_chance'] ) - 1 ) ) , 0 ) # home_ml_prediction
            predi_df.loc[away_filt, 'Away_mlPred'] = np.round( ( ( 1/matchup['away_chance'] ) * 100 ) - 100 , 0 ) # away_ml_prediction
        else: # home team is the underdog
            predi_df.loc[away_filt, 'Away_mlPred'] = np.round( (-1) * ( 100 / ( (1/matchup['away_chance'] ) - 1 ) ) , 0 ) # away_ml_prediction
            predi_df.loc[home_filt, 'Home_mlPred'] = np.round( ( ( 1/matchup['home_chance'] ) * 100 ) - 100 , 0 ) # home_ml_prediction

    # save our predictions 
    predi_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv', index=False)
    print("Moneyline Odds Predictions Saved!")

CalcMLOdds()
predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')
predi_df.loc[filt, ['AwayTeam', 'Away_xG', 'Away_mlPred', 'HomeTeam', 'Home_xG', 'Home_mlPred']]

Moneyline Odds Predictions Saved!


,AwayTeam,Away_xG,Away_mlPred,HomeTeam,Home_xG,Home_mlPred
0,Edmonton Oilers,3.15,-117.0,Buffalo Sabres,2.88,121.0
1,Ottawa Senators,2.65,-161.0,Chicago Blackhawks,1.98,163.0
2,Calgary Flames,2.15,116.0,Dallas Stars,2.32,-113.0
3,Washington Capitals,2.41,101.0,Los Angeles Kings,2.43,-101.0
4,Nashville Predators,2.78,-122.0,Vancouver Canucks,2.45,127.0
5,San Jose Sharks,2.20,132.0,Winnipeg Jets,2.55,-128.0


In [210]:
# Clean up our predictions !!! VALUABLE
predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv') # load our predictions data
predi_df.tail()

today = TodaysDateToString()

filt = ( predi_df['Date'] == today ) 

predi_df.drop(index=predi_df[filt].index, inplace=True) # remove any rows with today's date from our predictions dataset

predi_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv', index=False)

In [58]:
# = = = The Odds API = = = #
# This should pull odds for today's matchups from an API, we will need to capture the data we want, store it where we want it, and then use it

import requests
import json

# Request NHL odds data from the odds API, and return a list of list objects that each contain two dictionaries.  One for each team and their vegas odds
def PullVegasOdds():

    key = 'fa8219191b40b3fec426e43e1a7a37b8'
    url = f'https://api.the-odds-api.com/v4/sports/icehockey_nhl/odds?regions=us&oddsFormat=american&apiKey={key}&markets=h2h&bookmakers=draftkings'

    # Send a request to our API url
    response = requests.get(url)

    # Check to make sure we received a valid response
    if response.ok:
        pass
    else:
        print("Invalid request")

    # Parse our response into json format
    jsn = json.loads(response.text)

    # For each item in our response, I want to pull the expected outcomes and return a list of teams and their odds
    dk_predictions = []
    for row in jsn:
        try:
            #print(row['bookmakers'][0]['markets'][0]['outcomes'])
            #predictions.append({'AwayTeam': row['bookmakers'][0]['markets'][0]['outcomes'][0]['name'], 'Odds': row['bookmakers'][0]['markets'][0]['outcomes'][0]['price'] })
            dk_predictions.append(row['bookmakers'][0]['markets'][0]['outcomes'])
        except:
            pass

    return dk_predictions

#print(PullVegasOdds())

In [81]:
predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')
date_filt = ( predi_df['Date'] == today ) 
predi_df.loc[date_filt, 'Away_vegasPred'].notnull()

0    True
1    True
2    True
3    True
4    True
5    True
Name: Away_vegasPred, dtype: bool

In [83]:
# = = = Add each matchup's predicted odds to our predictions database = = = #

def UpdatePredictionsWithVegasOdds():
    
    # filter to check our predictions for today's matchups
    today = TodaysDateToString()
    
    # Create a filter to check our predictions database for today's games
    predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')
    date_filt = ( predi_df['Date'] == today ) 

    # Check if any of our vegas odds have not been updated for today
    # If they are all updated, do not run anything; return a statement
    if predi_df.loc[date_filt, 'Away_vegasPred'].notnull().any() or predi_df.loc[date_filt, 'Home_vegasPred'].notnull().any():
        return print("You've already updated today's games with Vegas Odds")
    else: # If they have not been updated, pull odds from the api and return an object with today's teams and their odds
        veg_odds = PullVegasOdds()

    # Re-Load predictions.csv
    predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')

    # For each matchup, update the away/home teams' vegas odds
    for matchup in veg_odds:
        
        # Grab the values we want from each matchup object
        teamA = matchup[0]['name']
        teamA_odds = matchup[0]['price']
        teamB = matchup[1]['name']
        teamB_odds = matchup[1]['price']

        # Update each team in today's matchups with vegas' odds, depending on whether they're home or away
        if teamA in predi_df.loc[date_filt, 'AwayTeam'].values:
            teamA_away_filt = ( ( predi_df['AwayTeam'] == teamA ) & ( predi_df['Date'] == today ) ) # filter where teamA is the away team for today's game
            teamB_home_filt = ( ( predi_df['HomeTeam'] == teamB ) & ( predi_df['Date'] == today ) ) # filter where teamB is the home team for today's game
            predi_df.loc[teamA_away_filt, 'Away_vegasPred'] = teamA_odds # Update away vegas odds 
            predi_df.loc[teamB_home_filt, 'Home_vegasPred'] = teamB_odds # Update home vegas odds
        elif teamA in predi_df.loc[date_filt, 'HomeTeam'].values:
            teamB_away_filt = ( ( predi_df['AwayTeam'] == teamB ) & ( predi_df['Date'] == today ) ) # filter where teamB is the away team for today's game
            teamA_home_filt = ( ( predi_df['HomeTeam'] == teamA ) & ( predi_df['Date'] == today ) ) # filter where teamA is the home team for today's game
            predi_df.loc[teamB_away_filt, 'Away_vegasPred'] = teamB_odds # Update away vegas odds
            predi_df.loc[teamA_home_filt, 'Home_vegasPred'] = teamA_odds # Update home vegas odds
        else:
            print(f"{teamA} is not in today's home teams or away teams")

    # Save predictions.csv
    predi_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv', index=False)

UpdatePredictionsWithVegasOdds()
predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')
print("============================")
print("Games for:", str(TodaysDateToString() ) )
predi_df[['AwayTeam', 'Away_xG', 'Away_mlPred', 'Away_vegasPred', 'HomeTeam', 'Home_xG', 'Home_mlPred', 'Home_vegasPred']]


You've already updated today's games with Vegas Odds
Games for: 3/6/23


,AwayTeam,Away_xG,Away_mlPred,Away_vegasPred,HomeTeam,Home_xG,Home_mlPred,Home_vegasPred
0,Edmonton Oilers,3.15,-117.0,-155.0,Buffalo Sabres,2.88,121.0,140.0
1,Ottawa Senators,2.65,-161.0,-260.0,Chicago Blackhawks,1.98,163.0,220.0
2,Calgary Flames,2.15,116.0,120.0,Dallas Stars,2.32,-113.0,-140.0
3,Washington Capitals,2.41,101.0,125.0,Los Angeles Kings,2.43,-101.0,-145.0
4,Nashville Predators,2.78,-122.0,-110.0,Vancouver Canucks,2.45,127.0,-110.0
5,San Jose Sharks,2.20,132.0,175.0,Winnipeg Jets,2.55,-128.0,-205.0


In [123]:
# = = = Feed Predictions = = = #

# Give me all picks today who have value
# Value: vegas:+200 & predicted:+150 or vegas:-1000 & predicted:-500 or vegas:-120 & predicted:+105

def DisplayTodaysPicks():
    # Open our predictions database
    predi_df = pd.read_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/predictions.csv')

    # apply the logic we desire to our away teams
    filtered_away_df = predi_df[ ( ( predi_df['Away_mlPred'] < predi_df['Away_vegasPred'] ) & ( predi_df['Away_mlPred'] > 0 ) & ( predi_df['Away_vegasPred'] < 0 ) ) | ( ( predi_df['Away_mlPred'] > predi_df['Away_vegasPred'] ) & ( ( predi_df['Away_mlPred'] < 0 ) & ( predi_df['Away_vegasPred'] < 0) ) | ( ( predi_df['Away_mlPred'] > 0 ) & ( predi_df['Away_vegasPred'] < 0 ) ) ) ]
    filtered_away_df = filtered_away_df[['AwayTeam', 'Away_mlPred', 'Away_vegasPred']]

    # apply the logic we desire to our home teams
    filtered_home_df = predi_df[ ( ( predi_df['Home_mlPred'] < predi_df['Home_vegasPred'] ) & ( predi_df['Home_mlPred'] > 0 ) & ( predi_df['Home_vegasPred'] < 0 ) ) | ( ( predi_df['Home_mlPred'] > predi_df['Home_vegasPred'] ) & ( ( predi_df['Home_mlPred'] < 0 ) & ( predi_df['Home_vegasPred'] < 0) ) | ( ( predi_df['Home_mlPred'] > 0 ) & ( predi_df['Home_vegasPred'] < 0 ) ) ) ]
    filtered_home_df = filtered_home_df[['HomeTeam', 'Home_mlPred', 'Home_vegasPred']]

    # combine our filtered picks
    todays_picks = pd.concat([filtered_away_df, filtered_home_df])

    # display our picks 
    todays_picks.style.set_precision(0)

/var/folders/w4/79twh2kd36j7vk84ch7bz3bw0000gn/T/ipykernel_54447/181510615.py:11: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  todays_picks.style.set_precision(0)


,AwayTeam,Away_mlPred,Away_vegasPred,HomeTeam,Home_mlPred,Home_vegasPred
0,Edmonton Oilers,-117,-155,nan,nan,nan
1,Ottawa Senators,-161,-260,nan,nan,nan
2,nan,nan,nan,Dallas Stars,-113,-140
3,nan,nan,nan,Los Angeles Kings,-101,-145
4,nan,nan,nan,Vancouver Canucks,127,-110
5,nan,nan,nan,Winnipeg Jets,-128,-205


In [108]:
def remove_decimal(val):
    return str(val).rstrip('.0')

In [68]:
# !! USED | WILL NOT NEED AGAIN !!

# Create a blank dataframe that will house our poisson distribution that will be used to calculate the odds through chance of victory
pssn = {
    'num':[],
    '0':[],
    '1':[],
    '2':[],
    '3':[],
    '4':[],
    '5':[],
    '6':[],
    '7':[],
    '8':[],
    '9':[],
    '10':[]
}

pssn_df = pd.DataFrame(pssn)
# Can I use iterrows() to plug my index in with the column?
# I want to iterate over columns and rows so that I can plug both into my algorithm
for i in range(11):
    pssn_df = pssn_df.append({'num':f'{i}'}, ignore_index=True)
pssn_df.to_csv('/Users/anthonyperpetua/Desktop/development/NHLpredictor/poissontable.csv')

/var/folders/w4/79twh2kd36j7vk84ch7bz3bw0000gn/T/ipykernel_43238/1411623760.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pssn_df = pssn_df.append({'num':f'{i}'}, ignore_index=True)
/var/folders/w4/79twh2kd36j7vk84ch7bz3bw0000gn/T/ipykernel_43238/1411623760.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pssn_df = pssn_df.append({'num':f'{i}'}, ignore_index=True)
/var/folders/w4/79twh2kd36j7vk84ch7bz3bw0000gn/T/ipykernel_43238/1411623760.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pssn_df = pssn_df.append({'num':f'{i}'}, ignore_index=True)
/var/folders/w4/79twh2kd36j7vk84ch7bz3bw0000gn/T/ipykernel_43238/1411623760.py:21: FutureWarning: The frame.append method is deprecated and will be removed from panda